In [1]:
from ipycanvas import Canvas

In [2]:
import dinkum
from dinkum.vfg import Gene
from dinkum.vfn import Tissue
from dinkum import Timecourse
from dinkum import observations

In [66]:
class MultiTissuePanel:
    def __init__(self):
        pass

class TissueActivityPanel:
    box_size = 25
    box_spacing = 5
    
    box_x_start = 100
    box_y_start = 50
    
    active_color = "DeepSkyBlue"
    inactive_color = "DarkGrey"
    
    def __init__(self, *, states=None):
        times = []
        all_gene_names = set()
        for (tp, state) in states:
            times.append(tp)
            for tissue in state.tissues:
                activity = state[tissue]
                all_gene_names.update(activity.genes_by_name)
        self.gene_names = list(sorted(all_gene_names))
        self.times = times
        self.states = states

    def estimate_panel_size(self):
        height = len(self.times) * (self.box_size + self.box_spacing) + self.box_y_start
        width = len(self.gene_names) * (self.box_size + self.box_spacing) + self.box_x_start
        return width, height
    
    def draw(self, tissue_name, is_active):
        width, height = self.estimate_panel_size()
        canvas = Canvas(width=width, height=height)
        self.draw_tissue(canvas, tissue_name, is_active)
        
        return canvas

    def draw_tissue(self, canvas, tissue_name, is_active_fn, *, x_offset=0):
        gene_names = self.gene_names
        times = self.times

        box_total_size = self.box_size + self.box_spacing

        for row in range(0, len(times)):
            for col in range(0, len(gene_names)):
                if is_active_fn(tissue_name, times[row], gene_names[col]):
                    canvas.fill_style = self.active_color
                else:
                    canvas.fill_style = self.inactive_color

                xpos = self.box_x_start + box_total_size*col
                ypos = self.box_y_start + box_total_size*row
                canvas.fill_rect(xpos, ypos, self.box_size, self.box_size)

        canvas.font = "18px Arial"
        canvas.text_baseline = "top"
        canvas.fill_style = "black"

        # row names / time points
        canvas.text_align = "right"
        for row in range(0, len(times)):
            xpos = self.box_x_start - box_total_size / 2
            ypos = self.box_y_start + box_total_size*row
            canvas.fill_text(times[row], xpos, ypos)

        # col names / genes
        canvas.text_align = "center"
        for col in range(0, len(gene_names)):
            ypos = self.box_y_start - box_total_size
            xpos = self.box_x_start + box_total_size*col + box_total_size / 2

            canvas.fill_text(gene_names[col], xpos, ypos, max_width = box_total_size)


In [67]:
def tc_record_activity(*, start=0, stop=10, gene_names=None, verbose=False):    
    tc = dinkum.Timecourse()

    state_record = []     # (tp_name, state)

    time_points = {}      # time_point_name => index
    all_tissues = set()   # all tissues across all time points

    for n, state in enumerate(tc.iterate(start=start, stop=stop)):
        tp = f"t={state.time}"
        if verbose:
            print(tp)
        times.append(tp)
        time_points[tp] = n

        for ti in state.tissues:
            all_tissues.add(ti.name)
            present = state[ti]
            if verbose:
                print(f"\ttissue={ti.name}, {present.report_activity()}")

        state_record.append((tp, state))

    def is_active(tissue_name, time_point, gene):
        time_idx = time_points[time_point]
        ga = states[time_idx]
        active = ga[1].get_by_tissue_name(tissue_name).is_active(gene)
        return bool(active)
    
    return state_record, list(tissues), is_active

In [69]:
dinkum.reset()

# set it all up!                                                            
x = Gene(name='X')
y = Gene(name='Y')
z = Gene(name='Z')

y.activated_by(source=x)
z.activated_by_and(sources=[x, y])

m = Tissue(name='M')
x.is_present(where=m, start=1)

states, tissues, is_active_fn = tc_record_activity(stop=5)

panel = TissueActivityPanel(states=states)
panel.draw('M', is_active_fn)

Canvas(height=200, width=190)

In [70]:
dinkum.reset()

x = Gene(name='X')
y = Gene(name='Y')
z = Gene(name='Z')

y.activated_by(source=x)
z.and_not(activator=x, repressor=y)

m = Tissue(name='M')
x.is_present(where=m, start=1)

# run!                                                                      
states, tissues, is_active_fn = tc_record_activity(stop=5)
panel = TissueActivityPanel(states=states)
panel.draw('M', is_active_fn)

Canvas(height=200, width=190)

In [75]:
dinkum.reset()

# set it all up!                                                            
x = Gene(name='X')
y = Gene(name='Y')
z = Gene(name='Z')
s = Gene(name='S')          # switches spec states b/t tissues M and N      

y.activated_by_and(sources=[x, s])
z.and_not(activator=x, repressor=y)

m = Tissue(name='M')
x.is_present(where=m, start=1)

m = Tissue(name='N')
x.is_present(where=m, start=1)
s.is_present(where=m, start=1)

# run!                                                                      
states, tissues, is_active_fn = tc_record_activity(stop=5)
panel = TissueActivityPanel(states=states)
panel.draw('M', is_active_fn)

Canvas(height=200, width=220)